In [3]:
mydatadir = "/data1/peterr/bilingual_data_for_ABClf"
outdir = "/data1/peterr/bilingual_data_for_ABClf_output"
import os
import pandas as pd
from ABClf import load_lexicon, get_variant, count_variants

lex = load_lexicon()
lex_balanced = load_lexicon(balanced=True)

import parse
def get_domain(url:str) -> str:
    pattern = "{protocol}://{domain}/{rest}"
    p = parse.compile(pattern)
    modified_pattern = "{protocol}://{domain}/"
    mp = parse.compile(modified_pattern)
    try:
        parse_result = p.parse(url)
        domain = parse_result["domain"]
    except TypeError:
        parse_result = mp.parse(url)
        domain = parse_result["domain"]
    return domain




for file in os.listdir(mydatadir):
    endpath = os.path.join(
            outdir,
            file+"ABClf_output.csv"
        )
    if os.path.basename(endpath) in os.listdir(outdir):
        continue
    if "en-mk" in file:
        kwargs = dict(nrows=1200000)
    else:
        kwargs = dict()
    df = pd.read_csv(
        os.path.join(
            mydatadir,
            file,
        ),
        sep="\t",
        error_bad_lines=False,
        warn_bad_lines=False,
        **kwargs
    ).iloc[:, [0,2]]

    df.columns=["url", "text"]

    s = df.groupby("url")["text"].apply(" ".join)
    

    ndf = pd.DataFrame(s)
    ndf["variant_unbalanced"] = ndf.text.apply(lambda s: get_variant(s, lex=lex))
    ndf["per_token_unbalanced"] = ndf.text.apply(lambda s: count_variants(s, lex)[1])
    ndf["variant_balanced"] = ndf.text.apply(lambda s: get_variant(s, lex=lex_balanced))
    ndf["per_token_balanced"] = ndf.text.apply(lambda s: count_variants(s, lex_balanced)[1])
    ndf["domain"] = [get_domain(url) for url in ndf.index.values]

    ndf.to_csv(
        endpath
    )
    

/home/peterr/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/home/peterr/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
